In [43]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import cross_val_score
import math

In [44]:
data = pd.read_csv("../raw_data/data_final.csv")

In [45]:
data.head(3)

,steam_appid,name,top_5_tags,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,...,topic_15,topic_16,topic_17,topic_18,topic_19,english,price,has_a_website,nb_review,owner_estimated
0,10,Counter-Strike,action fps multiplayer shooter classic,0.015903,0.015903,0.015903,0.015903,0.697844,0.015903,0.015903,...,0.015903,0.015903,0.015903,0.015903,0.015903,1,7.19,0,127873,15000000.0
1,30,Day of Defeat,fps world_war_ii multiplayer action shooter,0.015969,0.015969,0.015969,0.015969,0.696585,0.015969,0.015969,...,0.015969,0.015969,0.015969,0.015969,0.015969,1,3.99,1,3814,7500000.0
2,50,Half-Life: Opposing Force,fps action sci_fi singleplayer classic,0.015922,0.015922,0.015922,0.015922,0.697483,0.015922,0.015922,...,0.015922,0.015922,0.015922,0.015922,0.015922,1,3.99,0,5538,7500000.0


In [46]:
obj = data.isnull().sum()
for key,value in obj.iteritems():
    print(f'"{key}"')

"steam_appid"
"name"
"top_5_tags"
"topic_0"
"topic_1"
"topic_2"
"topic_3"
"topic_4"
"topic_5"
"topic_6"
"topic_7"
"topic_8"
"topic_9"
"topic_10"
"topic_11"
"topic_12"
"topic_13"
"topic_14"
"topic_15"
"topic_16"
"topic_17"
"topic_18"
"topic_19"
"english"
"price"
"has_a_website"
"nb_review"
"owner_estimated"


In [47]:
X = data.drop(columns=["steam_appid",
"name",
                       "top_5_tags",
                       "nb_review",
                       "owner_estimated"])
#X = data[['name']]
y = data['nb_review']

In [48]:
y

0        127873
1          3814
2          5538
3         28855
4         13559
          ...  
26862         3
26863         9
26864         1
26865         2
26866         4
Name: nb_review, Length: 26867, dtype: int64

In [49]:
X

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,...,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,english,price,has_a_website
0,0.015903,0.015903,0.015903,0.015903,0.697844,0.015903,0.015903,0.015903,0.015903,0.015903,...,0.015903,0.015903,0.015903,0.015903,0.015903,0.015903,0.015903,1,7.19,0
1,0.015969,0.015969,0.015969,0.015969,0.696585,0.015969,0.015969,0.015969,0.015969,0.015969,...,0.015969,0.015969,0.015969,0.015969,0.015969,0.015969,0.015969,1,3.99,1
2,0.015922,0.015922,0.015922,0.015922,0.697483,0.015922,0.015922,0.015922,0.015922,0.015922,...,0.015922,0.015922,0.015922,0.015922,0.015922,0.015922,0.015922,1,3.99,0
3,0.015922,0.015922,0.015922,0.015922,0.697483,0.015922,0.015922,0.015922,0.015922,0.015922,...,0.015922,0.015922,0.015922,0.015922,0.015922,0.015922,0.015922,1,7.19,1
4,0.015855,0.015855,0.015855,0.015855,0.698752,0.015855,0.015855,0.015855,0.015855,0.015855,...,0.015855,0.015855,0.015855,0.015855,0.015855,0.015855,0.015855,1,7.19,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26862,0.015694,0.701810,0.015694,0.015694,0.015694,0.015694,0.015694,0.015694,0.015694,0.015694,...,0.015694,0.015694,0.015694,0.015694,0.015694,0.015694,0.015694,1,2.09,0
26863,0.016478,0.016478,0.519926,0.016478,0.183470,0.016478,0.016478,0.016478,0.016478,0.016478,...,0.016478,0.016478,0.016478,0.016478,0.016478,0.016478,0.016478,1,1.69,0
26864,0.015483,0.015483,0.015483,0.015483,0.015483,0.015483,0.015483,0.015483,0.015483,0.015483,...,0.015483,0.015483,0.015483,0.015483,0.015483,0.015483,0.705831,1,3.99,1
26865,0.015466,0.015466,0.015466,0.015466,0.015466,0.015466,0.015466,0.015466,0.015466,0.015466,...,0.015466,0.015466,0.015466,0.015466,0.015466,0.015466,0.706139,1,5.19,1


In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [58]:
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

array([[-0.30765837, -0.2503797 , -0.25164995, ...,  0.13930505,
        -0.15185301,  0.70975925],
       [-0.30338961, -0.24440627, -0.24611452, ...,  0.13930505,
        -0.17923319,  0.70975925],
       [ 1.11221149, -0.24535648, -0.24699505, ...,  0.13930505,
         0.72431287,  0.70975925],
       ...,
       [ 1.21600006,  4.23446433, -0.24959067, ...,  0.13930505,
         0.14932901,  0.70975925],
       [-0.30854469, -0.25161996, -0.25279926, ...,  0.13930505,
        -0.28875393,  0.70975925],
       [-0.3057292 , -0.24768014, -0.24914833, ...,  0.13930505,
        -0.17923319,  0.70975925]])

In [68]:
from tensorflow.keras import Sequential
model = Sequential()

ModuleNotFoundError: No module named 'tensorflow'

In [65]:
model.fit(X_train, y_train)

LinearRegression()

In [66]:
cv_results = cross_validate(model, X_test, y_test, cv=5, scoring='neg_mean_absolute_error')

In [67]:
cv_results['test_score']

array([-2139.25360533, -2562.40181524, -1949.04336261, -2472.6435006 ,
       -2095.94623258])

In [63]:
# Preprocessor (prepare the dataset)
num_transformer = make_pipeline(SimpleImputer(), StandardScaler())
cat_transformer = OneHotEncoder()

preproc = make_column_transformer(
    (num_transformer, make_column_selector(dtype_include=['float64'])),
    (cat_transformer, make_column_selector(dtype_include=['object','bool'])),
    remainder='passthrough')

# Add Estimator
pipe = make_pipeline(preproc, model)

# Train pipeline
pipe.fit(X_train,y_train)

# Make predictions
pipe.predict(X_test.iloc[0:2])

# Score model
pipe.score(X_test,y_test)

# Cross validate pipeline
cross_val_score(pipe, X_train, y_train, cv=5, scoring='r2').mean()

0.010373151849976914